# 3. Train

**Tensorboard**
- Input at command: tensorboard --logdir=./log
- Input at browser: http://127.0.0.1:6006

In [1]:
import time
import os
import pandas as pd

project_name = 'Dog_Breed_Identification'
step_name = 'Train'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

cwd = os.getcwd()
model_path = os.path.join(cwd, 'model')
print('model_path: ' + model_path)

run_name: Dog_Breed_Identification_Train_20171024_194136
model_path: D:\Kaggle\Dogs_vs_Cats_Redux_Kernels_Edition\model


In [19]:
import os
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

x_data = []
y_data = {}
# x_val = []
# y_val = {}
x_test = []

cwd = os.getcwd()
feature_cgg16 = os.path.join(cwd, 'model', 'feature_VGG16_{}.h5'.format(171023))
feature_cgg19 = os.path.join(cwd, 'model', 'feature_VGG19_{}.h5'.format(171023))
feature_resnet50 = os.path.join(cwd, 'model', 'feature_ResNet50_{}.h5'.format(171023))
feature_xception = os.path.join(cwd, 'model', 'feature_Xception_{}.h5'.format(171023))
feature_inception = os.path.join(cwd, 'model', 'feature_InceptionV3_{}.h5'.format(171023))
for filename in [feature_cgg16, feature_cgg19, feature_resnet50]:
    with h5py.File(filename, 'r') as h:
        x_data.append(np.array(h['train']))
        y_data = np.array(h['train_label'])
#         x_val.append(np.array(h['val']))
#         y_val = np.array(h['val_label'])
        x_test.append(np.array(h['test']))

print(x_data[0].shape)
print(type(y_data[0]))
print(y_data[:10])
x_data = np.concatenate(x_data, axis=1)
# y_train = np.concatenate(y_train, axis=0)
# x_val = np.concatenate(x_val, axis=1)
# y_val = np.concatenate(y_val, axis=0)
x_test = np.concatenate(x_test, axis=1)
print(x_data.shape)
print(len(y_data))
# print(x_val.shape)
# print(len(y_val))
print(x_test.shape)

(25000, 512)
<class 'numpy.int32'>
[0 0 0 0 0 0 0 0 0 0]
(25000, 3072)
25000
(25000, 3072)


In [20]:
from keras.utils.np_utils import to_categorical

y_data = to_categorical(y_data)
print(y_data.shape)

(25000, 2)


In [21]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.02)
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(24500, 3072)
(24500, 2)
(500, 3072)
(500, 2)


In [25]:
from keras.models import *
from keras.layers import *
from keras.optimizers import Adam

# model = Sequential()
# model.add(Dense(5000, input_shape=x_train.shape[1:]))
# model.add(Dropout(0.5))
# model.add(Dense(120, activation='softmax'))

# inputs = Input(x_train.shape[1:])
# x = inputs
# x = Dropout(0.5)(x)
# x = Dense(120, activation='softmax')(x)
# model = Model(inputs, x)

model = Sequential()
model.add(Dense(1000, input_shape=x_train.shape[1:]))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer=Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
from keras.callbacks import TensorBoard

log_path = os.path.join(model_path, run_name)
print('log_dir:' + log_path)
tensorBoard = TensorBoard(log_dir=log_path)

log_dir:D:\Kaggle\Dogs_vs_Cats_Redux_Kernels_Edition\model\Dog_Breed_Identification_Train_20171024_194136


In [27]:
hist = model.fit(x_train, y_train,
                 batch_size=2,
                 epochs=20, #Increase this when not on Kaggle kernel
                 verbose=2,  #1 for ETA, 0 for silent
                 validation_data=(x_val, y_val), 
                 callbacks=[tensorBoard])

Train on 24500 samples, validate on 500 samples
Epoch 1/20


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(hist.history['loss'], color='b')
plt.plot(hist.history['val_loss'], color='r')
plt.show()
plt.plot(hist.history['acc'], color='b')
plt.plot(hist.history['val_acc'], color='r')
plt.show()

In [ ]:
def saveModel(model, run_name):
    cwd = os.getcwd()
    modelPath = os.path.join(cwd, 'model')
    if not os.path.isdir(modelPath):
        os.mkdir(modelPath)
    weigthsFile = os.path.join(modelPath, run_name + '.h5')
    model.save(weigthsFile)
saveModel(model, run_name)

## Predict

In [ ]:
y_pred = model.predict(x_test, batch_size=128)
print(y_pred.shape)

In [ ]:
files = os.listdir(os.path.join(cwd, 'input', 'data_test', 'test'))
print(files[:10])

In [ ]:
df2 = pd.read_csv('.\\input\\sample_submission.csv')
n_test = len(df2)
print(df2.shape)

In [ ]:
for i in range(0, 120):
    df2.iloc[:,[i+1]] = y_pred[:,i]
df2.to_csv('.\\output\\pred.csv', index=None)

In [ ]:
print('Done !')